# Assignment

In this assignment we will train a multilayer perceptron model on the MNIST dataset, extending the linear (single-layer) classifier in the tutorial. 

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file

# Google Colab

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 35.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [30]:
import numpy as np, pandas as pd
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

As in the tutorial, data for this assignment will consist of the MNIST handwritten digit dataset. The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [3]:
# --- Download dataset
datasets.download(name='mnist')

# --- Prepare generators and model inputs
gen_train, _, client = datasets.prepare(name='mnist', custom_layers=True)

[ 2022-04-08 00:58:02 ] [====================] 100.000% : Iterating | 000001    

**Note**: There is no need to change the above code for this assignment.

# Training

In this assignment we will train a multilayer perceptron, e.g. a simple neural network with at least one hidden layer. Be creative; feel free to try various permutations of: 

* number(s) of hidden layer(s)
* size of hidden layer(s)
* learning rate
* training iterations 

### Define backbone model

In [48]:
# --- Define input
x = Input(shape=784, dtype='float32')

# --- Define model
h1 = layers.Dense(392, activation = 'relu')(x)
h2 = layers.Dense(196, activation = 'relu')(h1)
h3 = layers.Dense(98, activation = 'relu')(h2)

logits = layers.Dense(10)(h3)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [49]:
h1.shape

TensorShape([None, 392])

In [50]:
x.shape

TensorShape([None, 784])

In [53]:
xs, _ = next(gen_train)
scores = backbone.predict(xs['dat'])
print(scores)

[[-4.1020217  -1.9817257  -1.31981    ...  0.85563225 -1.4764551
   1.1321486 ]
 [-2.6544466  -0.7075732  -0.18475021 ... -0.09030063 -0.47467992
   1.2669866 ]
 [-3.615666   -1.2391286  -1.0396391  ... -0.53538364 -0.9380453
   1.2407821 ]
 ...
 [-3.8964999  -1.3493209  -0.51874185 ...  0.15907927 -1.4387618
   0.3236822 ]
 [-3.5502093  -0.27608797 -1.3470035  ... -0.09002981 -1.1843852
   1.8362406 ]
 [-4.179923   -1.5278878  -0.64713985 ... -0.42081437 -1.0986567
   0.9203526 ]]


### Define training model

In [55]:
# --- Define inputs
inputs = {
    'dat': Input(shape = (784,), name = 'dat'),
    'digit': Input(shape = (1,), name = 'digit')}

# --- Define model
logits = backbone(inputs['dat'])


# --- Define loss
sce = losses.SparseCategoricalCrossentropy(from_logits=True)
loss = sce(y_true =inputs['digit'], y_pred = logits)

# --- Define metric
acc = metrics.sparse_categorical_accuracy(y_true=inputs['digit'], y_pred=logits)

Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors:

In [56]:
# --- Create model
training = Model(inputs=inputs, outputs={'logits': logits, 'loss' : loss, 'acc' : acc})

# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(acc, name = 'acc')

### Compiling

Once the `training` model has been created, use the following to define an optimizer and compile:

In [57]:
# --- Define optimizer 
optimizer = optimizers.Adam(learning_rate=1e-2)

# --- Compile model
training.compile(optimizer=optimizer)

The model is now compiled and ready for training!

### Train the model

In [58]:
training.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=10)

Epoch 1/10
250/250 [==============================] - 23s 92ms/step - loss: 1.0696 - acc: 0.7824
Epoch 2/10
250/250 [==============================] - 24s 97ms/step - loss: 0.2830 - acc: 0.9138
Epoch 3/10
250/250 [==============================] - 23s 91ms/step - loss: 0.2483 - acc: 0.9254
Epoch 4/10
250/250 [==============================] - 23s 92ms/step - loss: 0.2179 - acc: 0.9348
Epoch 5/10
250/250 [==============================] - 23s 91ms/step - loss: 0.2091 - acc: 0.9379
Epoch 6/10
250/250 [==============================] - 23s 91ms/step - loss: 0.1920 - acc: 0.9418
Epoch 7/10
250/250 [==============================] - 23s 92ms/step - loss: 0.2034 - acc: 0.9407
Epoch 8/10
250/250 [==============================] - 23s 92ms/step - loss: 0.1771 - acc: 0.9456
Epoch 9/10
250/250 [==============================] - 23s 91ms/step - loss: 0.1950 - acc: 0.9419
Epoch 10/10
250/250 [==============================] - 23s 91ms/step - loss: 0.1747 - acc: 0.9490


# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `training.predict(...)` on the training data. 

In [45]:
arrs = client.get(rows=np.arange(60000))


In [24]:
arrs

{'xs': {'dat': array([[-2., -2., -2., ..., -2., -2., -2.],
         [-2., -2., -2., ..., -2., -2., -2.],
         [-2., -2., -2., ..., -2., -2., -2.],
         ...,
         [-2., -2., -2., ..., -2., -2., -2.],
         [-2., -2., -2., ..., -2., -2., -2.],
         [-2., -2., -2., ..., -2., -2., -2.]], dtype=float32),
  'digit': array([[5],
         [0],
         [4],
         ...,
         [5],
         [6],
         [8]], dtype=uint8)},
 'ys': {}}

In [59]:
# --- Calculate accuracy
arrs = client.get(rows=np.arange(60000))
outputs = training.predict(arrs['xs'])
pred = np.argmax(outputs['logits'], axis=1)
# --- Serialize as *.csv file
df = pd.DataFrame(index=client.db.fnames.index)
df['true'] = arrs['xs']['digit'][:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print cumulative model performance
df['corr'].mean()

0.9494833333333333

**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [28]:
# --- Serialize a model
fname = './wjhan_model.hdf5'
backbone.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooked file would be submitted under the name `panteater_notebook.ipynb` and his model file would be submitted under the name `panteater_model.hdf5`.